In [1]:
import requests
import re
import json
import pandas as pd
import time
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [2]:
# 1. Obtener el HTML de la página del ranking
load_dotenv()
url = os.getenv("TARGET_URL_GOVERNMENT_EXPENDITURE")
BASE_URL = os.getenv("BASE_URL")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza error si la petición falla

In [3]:
# 2. Parsear HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
# 3. Buscar la tabla principal y extraer los links
table = soup.find("table")
country_links = []


if table:
    rows = table.find_all("tr")
    for row in rows:
        cell = row.find("td")
        if cell and cell.a:
            relative_link = cell.a["href"]
            full_url = f"{BASE_URL}{relative_link}"
            country_links.append(full_url)

# Convertir la lista en DataFrame
df_links = pd.DataFrame(country_links, columns=["url"])

# Guardar como archivo Parquet
df_links.to_parquet("data/country_government_expenditure_links.parquet", index=False)

In [5]:
# 4. Mostrar resultado
print("Links encontrados:", df_links.size)

Links encontrados: 101


In [6]:
# configuracion necesaria para descargar datos

master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

sleep_duration = 5

In [7]:
# Cargar los países válidos desde el archivo parquet
df_countries = pd.read_parquet("data/countries_with_hpi.parquet")
valid_countries = set(df_countries["country"].unique())

In [8]:
start_time = time.time()
for url in country_links:
    # Extraer país desde el URL
    country = urlparse(url).path.strip("/").split("/")[0]

    # Verificar si el país está en la lista válida
    if country not in valid_countries:
        print(f"⏭️ País {country} no está en la lista, se omite.")
        continue

    try:
        response = requests.get(url, headers=headers)
        html = response.text

        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            df = pd.DataFrame(data[1:], columns=data[0])
            df.columns = ["Period", "Value"]
            df["Country"] = country

            df_pivot = df.pivot(index="Period", columns="Country", values="Value")
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")
    
    time.sleep(sleep_duration)

end_time = time.time()
total_sleep_time = len(valid_countries) * sleep_duration
effective_time = end_time - start_time - total_sleep_time

print(f"⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_sleep_time:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {effective_time:.2f} s")

⏭️ País Albania no está en la lista, se omite.
⏭️ País Algeria no está en la lista, se omite.
⏭️ País Argentina no está en la lista, se omite.
⏭️ País Armenia no está en la lista, se omite.
✅ Datos extraídos correctamente para Australia
✅ Datos extraídos correctamente para Austria
⏭️ País Azerbaijan no está en la lista, se omite.
⏭️ País Bahamas no está en la lista, se omite.
⏭️ País Belarus no está en la lista, se omite.
✅ Datos extraídos correctamente para Belgium
⏭️ País Bermuda no está en la lista, se omite.
⏭️ País Bolivia no está en la lista, se omite.
⏭️ País Bosnia-and-Herzegovina no está en la lista, se omite.
⏭️ País Botswana no está en la lista, se omite.
✅ Datos extraídos correctamente para Brazil
⏭️ País Brunei no está en la lista, se omite.
✅ Datos extraídos correctamente para Bulgaria
⏭️ País Cameroon no está en la lista, se omite.
✅ Datos extraídos correctamente para Canada
⏭️ País Cape-Verde no está en la lista, se omite.
✅ Datos extraídos correctamente para Chile
✅ Da

In [9]:
master_df.tail()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,Colombia,Croatia,Cyprus,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
Period,,,,,,,,,,,,,,,,,,,,,
Q4 2020,117.45,22.50,30.43,438.60,7.74,130.38,9295.38,59585.45,3.27,1.38,...,2.56,296.09,86366.0,69.38,363.68,21.58,690.05,226.85,971.33,124.11
Q4 2021,127.99,24.27,32.20,491.76,8.09,139.52,10152.78,64571.46,3.77,1.49,...,2.79,315.30,95766.2,72.50,385.63,22.39,781.09,291.52,1061.68,127.44
Q4 2022,136.82,25.08,36.85,550.52,9.75,149.81,11089.30,65687.23,4.20,1.61,...,2.92,319.61,103569.9,78.45,413.22,22.73,754.39,599.71,1143.10,132.56
Q4 2023,148.37,26.45,37.94,630.48,11.18,160.13,11974.47,77136.89,4.73,1.76,...,3.28,356.08,105614.8,82.40,441.49,23.90,737.29,1141.85,1203.80,144.59
Q4 2024,161.47,28.77,39.64,660.17,11.59,173.31,12467.10,86802.11,5.67,1.89,...,3.60,362.05,111135.2,85.91,453.26,24.27,790.15,1947.17,1274.93,156.07


In [10]:
# obtain paraguay url
template_url = country_links[0]
country = "Paraguay"

# Separar por "/"
parts = template_url.strip("/").split("/")

# Reemplazar el país (posición 3: después del dominio)

parts[3] = country

# Reconstruir la URL
custom_url = "/".join(parts) + "/"

In [12]:
# extract and include paraguay

response = requests.get(custom_url, headers=headers)
html = response.text

match = re.search(pattern, html, re.DOTALL)
if match:
    data_str = match.group(1)
    data = json.loads(data_str)

    df = pd.DataFrame(data[1:], columns=data[0])
    df.columns = ["Period", "Value"]
    df["Country"] = country

    df_pivot = df.pivot(index="Period", columns="Country", values="Value")
    master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

    print(f"✅ Datos extraídos correctamente para {country}")

✅ Datos extraídos correctamente para Paraguay


In [13]:
# Resetear el índice para separar "Period"
master_df = master_df.reset_index()
master_df.tail()

,Period,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,Colombia,Croatia,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
257,Q4 2020,117.45,22.50,30.43,438.60,7.74,130.38,9295.38,59585.45,3.27,...,296.09,86366.0,69.38,363.68,21.58,690.05,226.85,971.33,124.11,9386.67
258,Q4 2021,127.99,24.27,32.20,491.76,8.09,139.52,10152.78,64571.46,3.77,...,315.30,95766.2,72.50,385.63,22.39,781.09,291.52,1061.68,127.44,11165.60
259,Q4 2022,136.82,25.08,36.85,550.52,9.75,149.81,11089.30,65687.23,4.20,...,319.61,103569.9,78.45,413.22,22.73,754.39,599.71,1143.10,132.56,11156.13
260,Q4 2023,148.37,26.45,37.94,630.48,11.18,160.13,11974.47,77136.89,4.73,...,356.08,105614.8,82.40,441.49,23.90,737.29,1141.85,1203.80,144.59,11413.76
261,Q4 2024,161.47,28.77,39.64,660.17,11.59,173.31,12467.10,86802.11,5.67,...,362.05,111135.2,85.91,453.26,24.27,790.15,1947.17,1274.93,156.07,12498.37


In [14]:
# Separar "Q1 1990" en "Quarter" y "Year"
master_df[["Quarter", "Year"]] = master_df["Period"].str.extract(r"Q(\d)\s+(\d{4})").astype(int)
master_df.tail()

,Period,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,Colombia,Croatia,...,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay,Quarter,Year
257,Q4 2020,117.45,22.50,30.43,438.60,7.74,130.38,9295.38,59585.45,3.27,...,69.38,363.68,21.58,690.05,226.85,971.33,124.11,9386.67,4,2020
258,Q4 2021,127.99,24.27,32.20,491.76,8.09,139.52,10152.78,64571.46,3.77,...,72.50,385.63,22.39,781.09,291.52,1061.68,127.44,11165.60,4,2021
259,Q4 2022,136.82,25.08,36.85,550.52,9.75,149.81,11089.30,65687.23,4.20,...,78.45,413.22,22.73,754.39,599.71,1143.10,132.56,11156.13,4,2022
260,Q4 2023,148.37,26.45,37.94,630.48,11.18,160.13,11974.47,77136.89,4.73,...,82.40,441.49,23.90,737.29,1141.85,1203.80,144.59,11413.76,4,2023
261,Q4 2024,161.47,28.77,39.64,660.17,11.59,173.31,12467.10,86802.11,5.67,...,85.91,453.26,24.27,790.15,1947.17,1274.93,156.07,12498.37,4,2024


In [15]:
# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Quarter", "Year"]]
master_df = master_df[cols]

master_df.tail()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,Colombia,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
257,4,2020,117.45,22.50,30.43,438.60,7.74,130.38,9295.38,59585.45,...,296.09,86366.0,69.38,363.68,21.58,690.05,226.85,971.33,124.11,9386.67
258,4,2021,127.99,24.27,32.20,491.76,8.09,139.52,10152.78,64571.46,...,315.30,95766.2,72.50,385.63,22.39,781.09,291.52,1061.68,127.44,11165.60
259,4,2022,136.82,25.08,36.85,550.52,9.75,149.81,11089.30,65687.23,...,319.61,103569.9,78.45,413.22,22.73,754.39,599.71,1143.10,132.56,11156.13
260,4,2023,148.37,26.45,37.94,630.48,11.18,160.13,11974.47,77136.89,...,356.08,105614.8,82.40,441.49,23.90,737.29,1141.85,1203.80,144.59,11413.76
261,4,2024,161.47,28.77,39.64,660.17,11.59,173.31,12467.10,86802.11,...,362.05,111135.2,85.91,453.26,24.27,790.15,1947.17,1274.93,156.07,12498.37


In [17]:
# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [18]:
master_df.tail()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,Colombia,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
257,2,2024,156.31,27.03,37.45,545.21,9.65,161.12,11818.46,62335.87,...,345.79,114008.8,79.09,421.86,23.78,744.67,1465.73,1235.75,147.51,9957.51
258,3,2024,155.21,26.19,34.95,535.10,9.65,165.91,11518.06,58028.78,...,354.34,108693.2,72.50,414.51,23.95,856.67,1618.97,1258.75,149.82,10463.28
259,4,2024,161.47,28.77,39.64,660.17,11.59,173.31,12467.10,86802.11,...,362.05,111135.2,85.91,453.26,24.27,790.15,1947.17,1274.93,156.07,12498.37
260,1,2025,157.62,26.20,37.32,517.27,9.55,179.13,12137.12,51936.80,...,351.54,118271.4,73.77,412.74,24.36,747.59,1884.21,1287.48,157.65,9983.01
261,2,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,81.73,NaN,NaN,NaN,NaN,1297.43,NaN,NaN


In [19]:
# Guardar como archivo Parquet
master_df.to_parquet("data/government_expenditure_values.parquet", index=False)